In [14]:
import random
import copy
import math

rects = input("Rectangles to be covered: ")
    
string = """1
10
1 5 0 7 2 7 3 7 3 8 0 8
2 9 2 0 3 0 3 7 2 7 2 6 2 5 2 4 2 2 2 1 
3 5 0 6 1 6 2 6 2 7 0 7 
4 4 1 5 2 5 2 6 1 6 
5 4 1 4 2 4 2 5 1 5 
6 5 1 2 2 2 2 4 1 4 1 3 
7 6 0 3 1 3 1 4 1 5 1 6 0 6 
8 4 0 2 1 2 1 3 0 3 
9 5 0 1 2 1 2 2 1 2 0 2 
10 4 0 0 2 0 2 1 0 1"""

def convert(string): #dicionario em q keys são retangulos e values vertices
    ret = {}
    for line in string.split('\n')[2:]:
        elements = list(map(int, line.split()))
        ret[elements[0]] = [(elements[i], elements[i + 1]) for i in range(2, elements[1] * 2 + 1, 2)]
    return ret

def encode(data): #id individual aos vértices
    return {v : k for k, v in dict(enumerate(set(sum(data.values(), [])))).items()}

def reverse_dict(data, encoding): #dicionario com vertices como key
    ret = {}
    for k, V in data.items():
        for v in V:
            lookup = encoding[v]
            if lookup not in ret:
                ret[lookup] = []
            ret[lookup] += [k]
    return ret


initial  = convert(string)
encoding = encode(initial) #id de cada vértice
data     = reverse_dict(initial, encoding)

m = (max(data.values()))[0] #nr of rectangles
guards = [0] * m #to check if all rect are covered

if rects != '':
    rec = ''.join(filter(lambda i: i.isdigit(), rects))
    r2cover = []
    for i in rec:
        r2cover.append(int(i))
    m = len(r2cover) #nr of rectangles
    toremove = []
    for r in initial.keys():
        if r not in r2cover:
            toremove.append(r)
    for i in toremove:
        del initial[i]
    encoding = encode(initial) #id de cada vértice
    data = reverse_dict(initial, encoding)

Rectangles to be covered: 


In [15]:
#1. Greedy

def GetMaxLen(data): 
    return max((len(v) for v in data.values()))

def GetMinLen(initial): # Saber qual o numero minimo de vertices existentes nos retangulos
    return min((len(v) for v in initial.values()))

def DeleteV(dic, vertice): #remove already guarded rectangles
    copia = copy.deepcopy(dic)
    vertexR = copia.get(vertice)
    del copia[vertice]
    for v in copia.values():
        for r in vertexR:
            if r in v:
                v.remove(r)
    return copia

def DeleteR(dic, id): # remove already chosen rectangles
    copia = copy.deepcopy(dic) #{rec:[v,v,v,v]}
    del copia[id]
    return copia

def Greedy(dic, guards, G, seed, func1, func2): #G is starting point
    copia = copy.deepcopy(dic)
    guardscp = copy.deepcopy(guards)
    Gcp = copy.deepcopy(G)
    while sum(guardscp) != m:
        maxlen = func1(copia)
        vertices = [] #opções que vêem mais vertices não guardados
        for v, r in copia.items():
            if len(r) == maxlen:
                vertices.append(v)
        random.seed(seed)
        guard = random.choice(vertices)
        Gcp += [dict(enumerate(set(sum(initial.values(), []))))[guard]] #para retornar as coordenadas
        for r in copia.get(guard):
            guardscp[r-1] = 1
        copia = func2(copia, guard)    
    return Gcp

GreedyA = Greedy(data, guards, [], None, GetMaxLen, DeleteV)

def Greedy_2(dic, dic2, G2, guards, func1, func2,func3,func4, int):
    copia = copy.deepcopy(dic) #{rec:[v,v,v,v]}
    copia2 = copy.deepcopy(dic2) #{V:[rec,rec,rec]}
    guardscp = copy.deepcopy(guards)
    Gcp2=copy.deepcopy(G2)
    while sum(guardscp) != m:
        newdic = {} #{V:[rec,rec,rec]}
        maxlen=0
        rectangles = []
        minlen = func1(copia) #Para ver o minimo de vertices que existe no retangulo
        for r, v  in copia.items():
            if len(v)==minlen:
                rectangles.append(r)
        guard_2= random.choice(rectangles) #escolhe random um dos retangulos com 4 vertices
        for v in copia.get(guard_2):
            idV = encoding[v]
            newdic[idV] = copia2[idV] #{V[r,r,r], V[r], V[r,r]...}...em relação ao retangulo escolhido(guard_2)
        maxlen=func3(newdic)
        vertices_2 = []
        copia = func4(copia,guard_2) #delete guarded rectangle
        for v, r in newdic.items():
            if len(r) == maxlen:
                vertices_2.append(v)
        guard2 = random.choice(vertices_2) #guarded vertice
        Rec = []
        for r,v in copia.items(): #For that deletes guarded rectangles
            for k in v:
                if(encoding[k]==guard2):
                    Rec += [r]
        for r in Rec:
            del copia[r]

        for v in copia.values(): #eliminate the vertice from all rectangles
            for k in v:
                if (encoding[k]==guard2):
                    v.remove(k)
    
        for r in copia2.get(guard2):
            guardscp[r-1] = 1
        Gcp2 += [dict(enumerate(set(sum(initial.values(), []))))[guard2]]
    return Gcp2

GreedyB = Greedy_2(initial, data, [], guards, GetMinLen, DeleteV,GetMaxLen,DeleteR,0) #escolher o retângulo ainda não coberto que tenha menos vértices incidentes e colocar um guarda num desses vértices que seja partilhado por mais retângulos ainda não cobertos

In [24]:
print("Greedy A: ")
print(GreedyA)

Greedy A: 
[(2, 1), (1, 6), (1, 4), (1, 2), (3, 7)]


In [23]:
print("Greedy B: ")
print(GreedyB)

Greedy B: 
[(2, 5), (1, 2), (2, 1), (2, 7), (1, 5)]


In [16]:
#3.a) Iterated local search - hill climbing

def SimpleHillClimbing(GreedySol, func1, func2, func3, func4, func5):
    current_state = copy.deepcopy(GreedySol)
    optimalg = math.ceil(m/3) #optimal number of guards
    count = 0
    while True:
        if len(current_state) == optimalg:
            break
        elif count > 100:
            break
        else:
            if m <= 150:
                neighbor_states = func1(current_state, data, guards, func3, func4, func5)
            else:
                neighbor_states = func2(current_state, data, guards)
            count += 1
            best_state = neighbor_states[0]
            for state in neighbor_states:
                if len(state) < len(best_state):
                    best_state = state
            if len(best_state) < len(current_state) :
                current_state = best_state
    return current_state

def RandomizedHillClimbing(GreedySol, func1, func2, func3, func4, func5, func6):
    current_state = copy.deepcopy(GreedySol)
    optimalg = math.ceil(m/3) #optimal number of guards
    count = 0
    while True:
        if len(current_state) == optimalg:
            break
        elif count > 100:
            break
        else:
            if m <= 150:
                neighbor_states = func1(current_state, data, guards, func4, func5, func6)
            else:
                neighbor_states = func2(current_state, data, guards)
            count += 1
            current_state = func3(neighbor_states)
    return current_state

def distributedProbability(neighbor_states): #for randomized Hill Climbing
    new_neighbor_states = []
    neighbor_states.sort(key=len)
    for i in range(len(neighbor_states)):
        new_neighbor_states.extend([neighbor_states[i]] * (len(neighbor_states) - i))
    return new_neighbor_states[random.randint(0, len(new_neighbor_states) - 1)]

#3.b) Simulated Annealing 

def SimulatedAnnealing(GreedySol, func1, func2, func3, func4, func5, func6):
    current_state = copy.deepcopy(GreedySol)
    optimalg = math.ceil(m/3) #optimal number of guards
    t = 1.0
    k = 0.995
    while t > 0.001:
        if len(current_state) == optimalg:
            break
        if m <= 150:
            neighbor_states = func1(current_state, data, guards, func4, func5, func6)
        else:
            neighbor_states = func2(current_state, data, guards)
        proposed_state = random.choice(neighbor_states)
        r = random.uniform(0,1)
        p = func3(proposed_state, current_state, t)
        if p >= r:
            current_state = proposed_state
        t = t * k
    return current_state
        

def prob(neighbor_state, current_state, t): #acceptance probability function for Simulated Annealing
    if len(neighbor_state) < len(current_state):
        prob = 1
    else:
        prob = math.exp(float(-(len(neighbor_state) - len(current_state)) / t))
    return prob

#Neighbor States

def NeighborStates(state, dic, guards, func1, func2, func3): #gives lists 3 vertexes short of optimal answers and runs it through greedy
    start = math.ceil(m/3) - 3 
    if start > 1:
        temp_state = random.sample(state, start)
    else:
        temp_state = [random.choice(state)]
    iteration = 10 # max number of neighbors
    copia = copy.deepcopy(dic)
    guardscopy = copy.deepcopy(guards)
    NS = []
    S = []
    for v in temp_state:
        idV = encoding[v]
        for r in copia.get(idV):
            guardscopy[r-1] = 1
        copia = func1(copia, idV)
    for i in range(iteration):
        S = func2(copia, guardscopy, temp_state, i, func3, func1)
        S.sort()
        if S not in NS:
            NS.append(S)
    if len(NS) < iteration: #tenta encontrar novas soluções vizinhas 
        dif = iteration - len(NS)
        for j in range(dif):
            S = func2(copia, guardscopy, temp_state, (iteration + j), func3, func1)
            S.sort()
            if S not in NS:
                NS.append(S)
    return NS

def SimpleNeighborStates(state, dic, guards): #1-flip
    n = max(dic.keys()) + 1 #nr of vertices
    guardlist = []
    iteration = 10
    copia = copy.deepcopy(dic)
    NS = []
    for i in state:
        idV = encoding[i]
        guardlist += [idV]
    nextoptions = []
    for i in [x for x in range(n) if x not in guardlist]:
        nextoptions += [i]
    for i in range(iteration):
        sol = copy.deepcopy(guardlist)
        nex = copy.deepcopy(nextoptions)
        guardscopy = copy.deepcopy(guards)
        change = random.choice(sol)
        sol.remove(change)
        for v in sol:
            for r in copia.get(v):
                guardscopy[r-1] = 1
        while sum(guardscopy) != m:
            nexto = random.choice(nex)
            nex.remove(nexto)
            sol += [nexto]
            for r in copia.get(nexto):
                guardscopy[r-1] = 1
        for i in range(len(sol)):
            sol[i] = dict(enumerate(set(sum(initial.values(), []))))[sol[i]]
        sol.sort()
        if sol not in NS:
            NS.append(sol)
    return NS
        
    
neighborstates = NeighborStates(GreedyA, data, guards, DeleteV, Greedy, GetMaxLen)
sneighborstates = SimpleNeighborStates(GreedyA, data, guards)
HillClimbing = SimpleHillClimbing(GreedyA, NeighborStates, SimpleNeighborStates, DeleteV, Greedy, GetMaxLen)
RandHillClimbing = RandomizedHillClimbing(GreedyA, NeighborStates, SimpleNeighborStates, distributedProbability, DeleteV, Greedy, GetMaxLen)
SimAnneal = SimulatedAnnealing(GreedyA, NeighborStates, SimpleNeighborStates, prob, DeleteV, Greedy, GetMaxLen)

In [25]:
print("Simple Hill Climbing: ")
print(HillClimbing)

Simple Hill Climbing: 
[(2, 1), (1, 6), (1, 4), (1, 2), (3, 7)]


In [26]:
print("Randomized Hill Climbing: ")
print(RandHillClimbing)

Randomized Hill Climbing: 
[(1, 2), (1, 5), (2, 1), (2, 7)]


In [27]:
print("Simulated Annealing: ")
print(SimAnneal)

Simulated Annealing: 
[(0, 1), (0, 6), (1, 2), (1, 5), (3, 7)]


In [28]:
#4.b) DFS com propagação de restrições (MAC with AC-3)

n = max(data.keys()) + 1 #nr of vertices

guardscheck = copy.deepcopy(guards)
guard = [''] * n #for propagation

graph = {k: list(range(n)) for k in range(n)}
for k, v in graph.items():
    v.remove(k)

graphcp = copy.deepcopy(graph)
datacp = copy.deepcopy(data)
initcp = copy.deepcopy(initial)

def dfs(graphcp, node, visited, G, guard, guardscheck, datacp, initcp, func1, func2):
    if sum(guardscheck) == m:
        return G
    if node == '':
        maxlen = func2(datacp)
        vertices = []
        for v, r in datacp.items():
            if len(r) == maxlen:
                vertices.append(v)
        node = random.choice(vertices)
    if node not in visited:
        guard[node] = 1      
        for r in data.get(node):
            guardscheck[r-1] = 1
        visited.append(node)
        G += [dict(enumerate(set(sum(initial.values(), []))))[node]]
        if sum(guardscheck) != m:
            for i in [x for x in range(n) if x not in visited]:
                if datacp.get(node) == datacp.get(i): #remove vertices that see the same rectangles
                    guard[i] = 0
                    visited.append(i)
                    coord = dict(enumerate(set(sum(initial.values(), []))))[i]
                    for k, v in initcp.items(): #propagação 
                        if coord in v:
                            initcp[k].remove(coord)
                        if len(initcp[k]) == 1:
                            idV = encoding[initcp[k][0]]
                            if idV not in visited:
                                guard[idV] = 1
                                for r in data.get(idV):
                                    guardscheck[r-1] = 1
                                visited.append(idV)
                                G += [dict(enumerate(set(sum(initial.values(), []))))[idV]]
                                datacp = func1(datacp, idV)
                    for j in range(n):
                        if j != i and (i in graphcp[j]):
                            graphcp[j].remove(i)
        datacp = func1(datacp, node)
    while sum(guardscheck) != m:
        maxlen = func2(datacp)
        best_options = []
        for i in range(maxlen + 1):
            for j in graphcp[node] and datacp.keys():
                if len(datacp.get(j)) == maxlen - i:
                    best_options.append(j)       
        for i in best_options:
            if i in visited: 
                continue
            dfs(graphcp, i, visited, G, guard, guardscheck, datacp, initcp, func1, func2)
    return G

DFS = dfs(graphcp, '', [], [], guard, guardscheck, datacp, initcp, DeleteV, GetMaxLen)

In [30]:
print("DFS com propagação de restrições: ")
print(DFS)

DFS com propagação de restrições: 
[(2, 4), (1, 6), (2, 1), (0, 7), (1, 2)]
